In [82]:
!pip install sklearn.metrics.plot_roc_curve

ERROR: Could not find a version that satisfies the requirement sklearn.metrics.plot_roc_curve (from versions: none)
ERROR: No matching distribution found for sklearn.metrics.plot_roc_curve


In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
sns.set(style="white")
import datetime
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.metrics import confusion_matrix
import datetime as dt


In [84]:
energy = pd.read_csv('demand_ens2.csv')
energy.head()

,Unnamed: 0,Date,zone,Energy Demand,Maximum Demand,ENS,Timestamp,longitud,latitud
0,0,2008-01-01,Antioquia,19934753.28,6543000,0.0,1.199146e+09,-75.520020,6.817353
1,1,2008-01-01,CQR,4939020.28,6543000,0.0,1.199146e+09,-75.728760,5.069058
2,2,2008-01-01,Centro,23350632.27,6543000,0.0,1.199146e+09,-72.905273,3.316018
3,3,2008-01-01,Costa Atlantica,23636908.26,6543000,0.0,1.199146e+09,-73.762207,10.509417
4,4,2008-01-01,Oriente,10794188.43,6543000,0.0,1.199146e+09,-72.531738,6.249776


In [85]:
energy['ENS_']=energy['ENS'].apply(lambda x: 1 if x > 0 else 0)

In [74]:
energy['zone'].unique()

array(['Antioquia', 'CQR', 'Centro', 'Costa Atlantica', 'Oriente', 'Sur',
       'THC', 'Valle'], dtype=object)

In [116]:
energy['Intercept'] = 1
region ={}
for x in energy['zone'].unique():
    region[x] = energy[energy['zone']==x]

In [87]:
ran_for = RandomForestRegressor()

In [88]:
ran_for_PRED = {}
for x in energy['zone'].unique():
    ran_for_P=ran_for.fit(region[x][['Energy Demand','Maximum Demand']],region[x]['ENS'])
    ran_for_PRED[x] = ran_for_P.predict(region[x][['Energy Demand','Maximum Demand']])

D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [89]:
prediction = []
for x in energy['zone'].unique():
    pred = pd.merge(region[x].reset_index(),pd.DataFrame(ran_for_PRED[x],columns=[['Regression']]),how='left',left_index=True,right_index=True)
    prediction.append(pred)

In [90]:
energy = pd.concat(prediction).drop(['index','Unnamed: 0'],axis=1).reset_index().drop(['index'],axis=1)

In [91]:
region ={}
ran_for_class = RandomForestClassifier()
ran_for_CLASS = {}
classifier = []

for x in energy['zone'].unique():
    region[x] = energy[energy['zone']==x]
    ran_for_C = ran_for_class.fit(region[x][['Energy Demand','Maximum Demand']],region[x]['ENS_'])
    ran_for_CLASS[x] = ran_for_C.predict_proba(region[x][['Energy Demand','Maximum Demand']])
    pred_class = pd.merge(region[x].reset_index(),pd.DataFrame(ran_for_CLASS[x][:,1],columns=[['Classification']]),how='left',left_index=True,right_index=True)
    classifier.append(pred_class)

#energy = pd.concat(prediction).drop(['index','Unnamed: 0'],axis=1).reset_index().drop(['index'],axis=1)
energy=pd.concat(classifier).drop(['index'],axis=1).reset_index().drop(['index'],axis=1)
energy.head()

D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,Date,zone,Energy Demand,Maximum Demand,ENS,Timestamp,longitud,latitud,ENS_,"(Regression,)","(Classification,)"
0,2008-01-01,Antioquia,19934753.28,6543000,0.0,1.199146e+09,-75.52002,6.817353,0,0.000,0.0
1,2008-01-02,Antioquia,25503208.08,8042000,0.0,1.199232e+09,-75.52002,6.817353,0,0.226,0.1
2,2008-01-03,Antioquia,26650538.37,8305000,0.0,1.199318e+09,-75.52002,6.817353,0,0.000,0.0
3,2008-01-04,Antioquia,26947028.39,8237000,0.0,1.199405e+09,-75.52002,6.817353,0,0.000,0.0
4,2008-01-05,Antioquia,25935570.09,7865000,0.0,1.199491e+09,-75.52002,6.817353,0,0.000,0.0


In [101]:
score_classifier = {}
for x in energy['zone'].unique():
    region[x] = energy[energy['zone']==x]
    ran_for_C = ran_for_class.fit(region[x][['Energy Demand','Maximum Demand']],region[x]['ENS_'])
    score_classifier[x] = ran_for_C.score(region[x][['Energy Demand','Maximum Demand']],region[x]['ENS_'])


In [115]:
sum(score_classifier.values())/8

0.763681592039801

In [117]:
ran_for = RandomForestRegressor()
score_regression = {}
for x in energy['zone'].unique():
    region[x] = energy[energy['zone']==x]
    X_train, X_test, Y_train,Y_test = train_test_split(region[x][['Intercept','Energy Demand','Maximum Demand']],region[x]['ENS_'],test_size=0.2)
    ran_for_C = ran_for.fit(X_train,Y_train)
    score_regression[x] = ran_for_C.score(X_test,Y_test)


D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [118]:
score_regression

{'Antioquia': -0.2576204123139503,
 'CQR': -0.22064252713011714,
 'Centro': -0.137234604574211,
 'Costa Atlantica': -0.22015773809523775,
 'Oriente': -0.25102075309426763,
 'Sur': 0.05069064311924887,
 'THC': 0.3722514985014984,
 'Valle': -0.3078154450261781}

In [92]:
energy.to_csv('Final Forecast.csv')

In [94]:
energy['ENS'].value_counts()

0.00      22831
1.00         57
7.19         31
2.80         29
2.00         26
          ...  
296.40        1
115.07        1
26.59         1
74.14         1
50.60         1
Name: ENS, Length: 4668, dtype: int64

In [ ]:
Antioquia = energy[energy['zone']==Antioquia]
X_train, X_test, Y_train,Y_test = train_test_split(energy_region[['Energy Demand','Maximum Demand']],energy_region['ENS_'],test_size=0.2)
ran_for = RandomForestClassifier()
ran_for=ran_for.fit(X_train,Y_train)
n_pred_train=ran_for.predict(X_train)
n_pred_test=ran_for.predict(X_test)
print(accuracy_score(Y_train,n_pred_train),accuracy_score(Y_test,n_pred_test))